# Anomaly detection project

In [1]:
# Ignoring warning messages from python
import warnings
warnings.filterwarnings('ignore')

# General use imports
import pandas as pd
import numpy as np


# Visualization imports
import matplotlib.pyplot as plt
import seaborn as sns

# Acquisition and preparation imports

# Classification performance measuring imports
from sklearn import metrics

# Question to answer:
   >### Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?

### *I. Checking out cohorts*

In [2]:
# Getting the data

cohorts = pd.read_csv('cohorts_data.csv')
cohorts.head()

,Unnamed: 0,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
0,0,1,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
1,1,2,Badlands,#badlands,2014-06-04,2014-08-22,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
2,2,3,Carlsbad,#carlsbad,2014-09-04,2014-11-05,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
3,3,4,Denali,#denali,2014-10-20,2015-01-18,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
4,4,5,Everglades,#everglades,2014-11-18,2015-02-24,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1


In [3]:
# Getting to know the data

cohorts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  53 non-null     int64  
 1   id          53 non-null     int64  
 2   name        53 non-null     object 
 3   slack       53 non-null     object 
 4   start_date  53 non-null     object 
 5   end_date    53 non-null     object 
 6   created_at  53 non-null     object 
 7   updated_at  53 non-null     object 
 8   deleted_at  0 non-null      float64
 9   program_id  53 non-null     int64  
dtypes: float64(1), int64(3), object(6)
memory usage: 4.3+ KB


In [4]:
# What are the unique categories in some of the columns
# For program_id

cohorts.program_id.nunique()

4

In [10]:
cohorts.program_id.unique()

array([1, 4, 2, 3])

In [5]:
# For id

cohorts.id.nunique()

53

In [6]:
cohorts.id.unique()

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  51,  52,  53,  55,  56,
        57,  58,  59,  60,  61,  62, 132, 133, 134, 135, 136, 137, 138,
       139])

In [7]:
# For name

cohorts.name.nunique()

53

In [8]:
cohorts.name.unique()

array(['Arches', 'Badlands', 'Carlsbad', 'Denali', 'Everglades',
       'Franklin', 'Glacier', 'Hampton', 'Apollo', 'Balboa', 'Ike',
       'Joshua', 'Kings', 'Lassen', 'Mammoth', 'Niagara', 'Olympic',
       'Pinnacles', 'Quincy', 'Redwood', 'Sequoia', 'Teddy', 'Ulysses',
       'Voyageurs', 'Wrangell', 'Xanadu', 'Yosemite', 'Staff', 'Zion',
       'Ada', 'Andromeda', 'Betelgeuse', 'Ceres', 'Bayes', 'Deimos',
       'Europa', 'Fortuna', 'Curie', 'Apex', 'Ganymede', 'Hyperion',
       'Darden', 'Io', 'Bash', 'Jupiter', 'Kalypso', 'Easley', 'Luna',
       'Marco', 'Placeholder for students in transition', 'Florence',
       'Neptune', 'Oberon'], dtype=object)

### *II. Checking out logs*

In [11]:
# Getting the 2nd set of data

logs = pd.read_csv('logs_data.csv')
logs.head()

,Unnamed: 0,date,time,path,user_id,cohort_id,ip
0,0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61
1,1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61
2,2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61
3,3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61
4,4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61


In [12]:
# Getting to know the data

logs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900223 entries, 0 to 900222
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  900223 non-null  int64  
 1   date        900223 non-null  object 
 2   time        900223 non-null  object 
 3   path        900222 non-null  object 
 4   user_id     900223 non-null  int64  
 5   cohort_id   847330 non-null  float64
 6   ip          900223 non-null  object 
dtypes: float64(1), int64(2), object(4)
memory usage: 48.1+ MB


In [13]:
# What are the unique categories in some of the columns
# For cohort_id

logs.cohort_id.nunique()

47

In [14]:
logs.cohort_id.unique()

array([  8.,  22.,  21.,   1.,  16.,  18.,  19.,  nan,  13.,  14.,  15.,
         7.,   4.,  12.,  17.,  23.,   2.,   9.,  11.,  24.,  25.,  26.,
         6.,  27.,  28.,  29.,  31.,  32.,  33.,  34.,  51.,  52.,  53.,
        55.,  56.,  57.,   5.,  58.,  59.,  61.,  62., 132., 134., 133.,
       135., 138., 137., 139.])

In [15]:
# For user_id

logs.user_id.nunique()

981

In [16]:
# For path

logs.path.nunique()

2313

In [17]:
# For ip

logs.ip.nunique()

5531

>### Takeaways
    
        * Cohorts:
            - There is the same number of cohort names and IDs (53), this means that all entries are unique because there are only 53 entries.
            - Unnamed, slack, created_at, updated_at, deleted_at, program_id will not be useful in identifying the lesson that was referred to the most
        * Logs
            - Unnamed will not be useful in identifying the lesson that was referred to the most

### *III. Creating a combined dataframe*

In [20]:
cohorts.head()

,Unnamed: 0,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
0,0,1,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
1,1,2,Badlands,#badlands,2014-06-04,2014-08-22,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
2,2,3,Carlsbad,#carlsbad,2014-09-04,2014-11-05,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
3,3,4,Denali,#denali,2014-10-20,2015-01-18,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
4,4,5,Everglades,#everglades,2014-11-18,2015-02-24,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1


In [21]:
# Dropping unnecessary columns and displaying the names of df

cohorts = cohorts.drop(columns = ['Unnamed: 0', 'slack', 'created_at', 'updated_at', 'deleted_at', 'program_id'])
cohorts.head()

,id,name,start_date,end_date
0,1,Arches,2014-02-04,2014-04-22
1,2,Badlands,2014-06-04,2014-08-22
2,3,Carlsbad,2014-09-04,2014-11-05
3,4,Denali,2014-10-20,2015-01-18
4,5,Everglades,2014-11-18,2015-02-24


In [22]:
logs.head()

,Unnamed: 0,date,time,path,user_id,cohort_id,ip
0,0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61
1,1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61
2,2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61
3,3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61
4,4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61


In [35]:
# Merging cohorts and logs

combined_df = pd.merge(logs, cohorts, how = 'outer', left_on = 'cohort_id', right_on= 'id')
combined_df.head()

,Unnamed: 0,date,time,path,user_id,cohort_id,ip,id,name,start_date,end_date
0,0.0,2018-01-26,09:55:03,/,1.0,8.0,97.105.19.61,8.0,Hampton,2015-09-22,2016-02-06
1,1.0,2018-01-26,09:56:02,java-ii,1.0,8.0,97.105.19.61,8.0,Hampton,2015-09-22,2016-02-06
2,2.0,2018-01-26,09:56:05,java-ii/object-oriented-programming,1.0,8.0,97.105.19.61,8.0,Hampton,2015-09-22,2016-02-06
3,3.0,2018-01-26,09:56:06,slides/object_oriented_programming,1.0,8.0,97.105.19.61,8.0,Hampton,2015-09-22,2016-02-06
4,58.0,2018-01-26,10:40:15,javascript-i/functions,1.0,8.0,97.105.19.61,8.0,Hampton,2015-09-22,2016-02-06


In [36]:
# Converting the datetime column to datetime type and dropping duplicated columns (date and time)

def convert_datetime(df):
    df['datetime'] = pd.to_datetime(df.date + ' ' + df.time)
    df = df.drop(columns= ['Unnamed: 0', 'date', 'time', 'id'])
    return df

In [37]:
combined_df = convert_datetime(combined_df)
combined_df.head()

,path,user_id,cohort_id,ip,name,start_date,end_date,datetime
0,/,1.0,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2018-01-26 09:55:03
1,java-ii,1.0,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2018-01-26 09:56:02
2,java-ii/object-oriented-programming,1.0,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2018-01-26 09:56:05
3,slides/object_oriented_programming,1.0,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2018-01-26 09:56:06
4,javascript-i/functions,1.0,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2018-01-26 10:40:15


In [38]:
# Reordering columns

combined_df[['datetime', 'cohort_id', 'name', 'user_id', 'path', 'start_date', 'end_date']]

,datetime,cohort_id,name,user_id,path,start_date,end_date
0,2018-01-26 09:55:03,8.0,Hampton,1.0,/,2015-09-22,2016-02-06
1,2018-01-26 09:56:02,8.0,Hampton,1.0,java-ii,2015-09-22,2016-02-06
2,2018-01-26 09:56:05,8.0,Hampton,1.0,java-ii/object-oriented-programming,2015-09-22,2016-02-06
3,2018-01-26 09:56:06,8.0,Hampton,1.0,slides/object_oriented_programming,2015-09-22,2016-02-06
4,2018-01-26 10:40:15,8.0,Hampton,1.0,javascript-i/functions,2015-09-22,2016-02-06
...,...,...,...,...,...,...,...
900224,NaT,NaN,Balboa,NaN,NaN,2015-11-03,2016-03-11
900225,NaT,NaN,Redwood,NaN,NaN,2017-07-31,2017-12-06
900226,NaT,NaN,Ada,NaN,NaN,2019-02-04,2019-06-16
900227,NaT,NaN,Io,NaN,NaN,2020-07-20,2021-01-21


### *1. Looking for other null values in the dataset*

In [43]:
# Displaying the number of nulls in the columns

combined_df.isnull().sum()

path              7
user_id           6
cohort_id     52899
ip                6
name          52893
start_date    52893
end_date      52893
datetime          6
dtype: int64

>**Displaying the nulls in each column to have a idea of what they correspond to in other columns**

In [44]:
combined_df[combined_df['path'].isnull()]

,path,user_id,cohort_id,ip,name,start_date,end_date,datetime
615525,NaN,586.0,55.0,72.177.240.51,Curie,2020-02-03,2020-07-07,2020-04-08 09:25:18
900223,NaN,NaN,NaN,NaN,Carlsbad,2014-09-04,2014-11-05,NaT
900224,NaN,NaN,NaN,NaN,Balboa,2015-11-03,2016-03-11,NaT
900225,NaN,NaN,NaN,NaN,Redwood,2017-07-31,2017-12-06,NaT
900226,NaN,NaN,NaN,NaN,Ada,2019-02-04,2019-06-16,NaT
900227,NaN,NaN,NaN,NaN,Io,2020-07-20,2021-01-21,NaT
900228,NaN,NaN,NaN,NaN,Placeholder for students in transition,2021-03-03,2029-03-01,NaT


In [45]:
combined_df[combined_df['user_id'].isnull()]

,path,user_id,cohort_id,ip,name,start_date,end_date,datetime
900223,NaN,NaN,NaN,NaN,Carlsbad,2014-09-04,2014-11-05,NaT
900224,NaN,NaN,NaN,NaN,Balboa,2015-11-03,2016-03-11,NaT
900225,NaN,NaN,NaN,NaN,Redwood,2017-07-31,2017-12-06,NaT
900226,NaN,NaN,NaN,NaN,Ada,2019-02-04,2019-06-16,NaT
900227,NaN,NaN,NaN,NaN,Io,2020-07-20,2021-01-21,NaT
900228,NaN,NaN,NaN,NaN,Placeholder for students in transition,2021-03-03,2029-03-01,NaT


In [46]:
combined_df[combined_df['cohort_id'].isnull()]

,path,user_id,cohort_id,ip,name,start_date,end_date,datetime
53122,/,48.0,NaN,97.105.19.61,NaN,NaN,NaN,2018-01-26 16:46:16
53123,spring/extra-features/form-validation,48.0,NaN,97.105.19.61,NaN,NaN,NaN,2018-01-26 16:46:24
53124,/,48.0,NaN,97.105.19.61,NaN,NaN,NaN,2018-01-26 17:54:24
53125,/,48.0,NaN,97.105.19.61,NaN,NaN,NaN,2018-01-26 18:32:03
53126,mysql/relationships/joins,48.0,NaN,97.105.19.61,NaN,NaN,NaN,2018-01-26 18:32:17
...,...,...,...,...,...,...,...,...
900224,NaN,NaN,NaN,NaN,Balboa,2015-11-03,2016-03-11,NaT
900225,NaN,NaN,NaN,NaN,Redwood,2017-07-31,2017-12-06,NaT
900226,NaN,NaN,NaN,NaN,Ada,2019-02-04,2019-06-16,NaT
900227,NaN,NaN,NaN,NaN,Io,2020-07-20,2021-01-21,NaT


In [47]:
combined_df[combined_df['ip'].isnull()]

,path,user_id,cohort_id,ip,name,start_date,end_date,datetime
900223,NaN,NaN,NaN,NaN,Carlsbad,2014-09-04,2014-11-05,NaT
900224,NaN,NaN,NaN,NaN,Balboa,2015-11-03,2016-03-11,NaT
900225,NaN,NaN,NaN,NaN,Redwood,2017-07-31,2017-12-06,NaT
900226,NaN,NaN,NaN,NaN,Ada,2019-02-04,2019-06-16,NaT
900227,NaN,NaN,NaN,NaN,Io,2020-07-20,2021-01-21,NaT
900228,NaN,NaN,NaN,NaN,Placeholder for students in transition,2021-03-03,2029-03-01,NaT


In [48]:
combined_df[combined_df['name'].isnull()]

,path,user_id,cohort_id,ip,name,start_date,end_date,datetime
53122,/,48.0,NaN,97.105.19.61,NaN,NaN,NaN,2018-01-26 16:46:16
53123,spring/extra-features/form-validation,48.0,NaN,97.105.19.61,NaN,NaN,NaN,2018-01-26 16:46:24
53124,/,48.0,NaN,97.105.19.61,NaN,NaN,NaN,2018-01-26 17:54:24
53125,/,48.0,NaN,97.105.19.61,NaN,NaN,NaN,2018-01-26 18:32:03
53126,mysql/relationships/joins,48.0,NaN,97.105.19.61,NaN,NaN,NaN,2018-01-26 18:32:17
...,...,...,...,...,...,...,...,...
106010,javascript-ii,717.0,NaN,136.50.102.126,NaN,NaN,NaN,2021-04-21 12:49:00
106011,javascript-ii/es6,717.0,NaN,136.50.102.126,NaN,NaN,NaN,2021-04-21 12:49:02
106012,javascript-ii/map-filter-reduce,717.0,NaN,136.50.102.126,NaN,NaN,NaN,2021-04-21 12:51:27
106013,javascript-ii/promises,717.0,NaN,136.50.102.126,NaN,NaN,NaN,2021-04-21 12:52:37


In [49]:
combined_df[combined_df['start_date'].isnull()]

,path,user_id,cohort_id,ip,name,start_date,end_date,datetime
53122,/,48.0,NaN,97.105.19.61,NaN,NaN,NaN,2018-01-26 16:46:16
53123,spring/extra-features/form-validation,48.0,NaN,97.105.19.61,NaN,NaN,NaN,2018-01-26 16:46:24
53124,/,48.0,NaN,97.105.19.61,NaN,NaN,NaN,2018-01-26 17:54:24
53125,/,48.0,NaN,97.105.19.61,NaN,NaN,NaN,2018-01-26 18:32:03
53126,mysql/relationships/joins,48.0,NaN,97.105.19.61,NaN,NaN,NaN,2018-01-26 18:32:17
...,...,...,...,...,...,...,...,...
106010,javascript-ii,717.0,NaN,136.50.102.126,NaN,NaN,NaN,2021-04-21 12:49:00
106011,javascript-ii/es6,717.0,NaN,136.50.102.126,NaN,NaN,NaN,2021-04-21 12:49:02
106012,javascript-ii/map-filter-reduce,717.0,NaN,136.50.102.126,NaN,NaN,NaN,2021-04-21 12:51:27
106013,javascript-ii/promises,717.0,NaN,136.50.102.126,NaN,NaN,NaN,2021-04-21 12:52:37


In [50]:
combined_df[combined_df['end_date'].isnull()]

,path,user_id,cohort_id,ip,name,start_date,end_date,datetime
53122,/,48.0,NaN,97.105.19.61,NaN,NaN,NaN,2018-01-26 16:46:16
53123,spring/extra-features/form-validation,48.0,NaN,97.105.19.61,NaN,NaN,NaN,2018-01-26 16:46:24
53124,/,48.0,NaN,97.105.19.61,NaN,NaN,NaN,2018-01-26 17:54:24
53125,/,48.0,NaN,97.105.19.61,NaN,NaN,NaN,2018-01-26 18:32:03
53126,mysql/relationships/joins,48.0,NaN,97.105.19.61,NaN,NaN,NaN,2018-01-26 18:32:17
...,...,...,...,...,...,...,...,...
106010,javascript-ii,717.0,NaN,136.50.102.126,NaN,NaN,NaN,2021-04-21 12:49:00
106011,javascript-ii/es6,717.0,NaN,136.50.102.126,NaN,NaN,NaN,2021-04-21 12:49:02
106012,javascript-ii/map-filter-reduce,717.0,NaN,136.50.102.126,NaN,NaN,NaN,2021-04-21 12:51:27
106013,javascript-ii/promises,717.0,NaN,136.50.102.126,NaN,NaN,NaN,2021-04-21 12:52:37


In [51]:
combined_df[combined_df['datetime'].isnull()]

,path,user_id,cohort_id,ip,name,start_date,end_date,datetime
900223,NaN,NaN,NaN,NaN,Carlsbad,2014-09-04,2014-11-05,NaT
900224,NaN,NaN,NaN,NaN,Balboa,2015-11-03,2016-03-11,NaT
900225,NaN,NaN,NaN,NaN,Redwood,2017-07-31,2017-12-06,NaT
900226,NaN,NaN,NaN,NaN,Ada,2019-02-04,2019-06-16,NaT
900227,NaN,NaN,NaN,NaN,Io,2020-07-20,2021-01-21,NaT
900228,NaN,NaN,NaN,NaN,Placeholder for students in transition,2021-03-03,2029-03-01,NaT


In [ ]:
combined_df[combined_df['ip'].isnull()]

## 2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?

In [ ]:
def value_counts_and_frequencies(s: pd.Series, dropna=True) -> pd.DataFrame:
    return pd.merge(
        s.value_counts(dropna=False).rename('count'),
        s.value_counts(dropna=False, normalize=True).rename('proba'),
        left_index=True,
        right_index=True)

In [ ]:
path_df = value_counts_and_frequencies(combined_df.path)
path_df.head()

In [ ]:
combined_df.head()

In [ ]:
combined_df.groupby('name').path.value_counts(normalize=True)

In [ ]:
# combined_df.groupby('name').path.value_counts(normalize=True).plot(kind='bar')

In [ ]:
combined_df.name.str.contains('Andromeda')

In [ ]:
combined_df.groupby('name'== ).path.value_counts(normalize=True)

In [ ]:
# def make_datetime_index(df):
#     df['datetime'] = pd.to_datetime(df.date + ' ' + df.time)
#     df = df.set_index('datetime').sort_index()
#     df = df.drop(columns= ['Unnamed: 0', 'date', 'time'])
#     return df

In [ ]:
# combined_df = make_datetime_index(combined_df)
# combined_df.head()